In [2]:
import os
import numpy as np
from pathlib import Path
import pandas as pd
import optuna
import xgboost as xgb
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline, make_union, FeatureUnion
from sklearn.preprocessing import   StandardScaler,RobustScaler, OneHotEncoder, FunctionTransformer, PowerTransformer, QuantileTransformer, \
                                    PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, RFE, RFECV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [21]:
class Config:
    CONTEST_TAG = 'playground-series-s3e24'
    def __init__(self):
        self.iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
        self.seed = 42
        self.n_jobs = -1
        self.splits = 5
        self.target_col = 'smoking'
        self.index_col = 'id'
        self.optuna_trials = 30
        if self.iskaggle:
            self.train_path = Path(f'/kaggle/input/{self.CONTEST_TAG}/train.csv')
            self.test_path = Path(f'/kaggle/input/{self.CONTEST_TAG}/test.csv')
            self.submission_path = Path(f'/kaggle/input/{self.CONTEST_TAG}/sample_submission.csv')
        else:
            self.train_path = Path(f'./input/{self.CONTEST_TAG}/train.csv')
            self.test_path = Path(f'./input/{self.CONTEST_TAG}/test.csv')
            self.submission_path = Path(f'./input/{self.CONTEST_TAG}/sample_submission.csv')

        self.default_rf_params =  {   
            'criterion': 'gini',
            'n_jobs': -1, 
            'random_state': self.seed
        }

CFG = Config()

In [15]:
cat_cols = ['Urine protein', 'hearing(left)', 'hearing(right)', 'dental caries']
num_cols = ['age', 'height(cm)', 'weight(kg)', 'waist(cm)', 'eyesight(left)',
       'eyesight(right)', 'systolic', 'relaxation', 'fasting blood sugar', 'Cholesterol', 'triglyceride',
       'HDL', 'LDL', 'hemoglobin', 'serum creatinine', 'AST', 'ALT', 'Gtp']

In [ ]:
train_df = pd.read_csv(CFG.train_path, index_col=CFG.index_col)
test_df = pd.read_csv(CFG.test_path, index_col=CFG.index_col)
X_train  = train_df.drop(CFG.target_col, axis=1)
y_train  = train_df[CFG.target_col]